In [ ]:
import pandas as pd
# import modin.pandas as pd
import numpy as np
import pickle as pkl
from os import listdir, walk
from os.path import isfile, join
from dask.distributed import Client
import dask.dataframe as dd

import datetime

import json
import time

import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

%matplotlib inline

In [ ]:
care_defs = pd.read_csv('/data/deidentified_trews_deployment_data/cdm/care_unit_defs.csv')
# SMH 3A L&D is a maternity unit. Not marked in the file
care_defs.at[care_defs.loc[care_defs.care_unit == 'SMH 3A L&D'].index[0], 'maternity'] = True

In [ ]:
pat_enc = pd.read_csv('/data/deidentified_trews_deployment_data/cdm/pat_enc.csv')

In [ ]:
df = dd.read_csv('/data/deidentified_trews_deployment_data/cdm/cdm_t.csv')

In [ ]:
# patient cohort where LOS (and labels) updated to be less than 15 days
cohort = pd.read_csv("/data/adarsh/fda_project_data/study_cohort_with_outcome_truncated_15days.csv")

In [ ]:
cohort['admit_time'] = pd.to_datetime(cohort.admit_time)

In [ ]:
ed_wrong_alert_ids = np.load("/data/adarsh/fda_project_data/predictions/ed_admits_wrong_alerts.npy")

In [ ]:
test_ids = np.load("/data/adarsh/fda_project_data/predictions/test_outcome_1_ids.npy")

In [ ]:
test_cohort = cohort.query('enc_id in @test_ids')

In [ ]:
# use dask default scheduling
start = time.time()

# find all rows in cdm_t corresponding to the study cohort
cdm_t = df.loc[df.enc_id.isin(test_ids)].compute()

end = time.time()
print((end-start)/60.)

## Find last department before event

In [ ]:
care_unit_df = cdm_t.query('fid == "care_unit"')

In [ ]:
care_unit_df = test_cohort.merge(care_unit_df[['enc_id', 'tsp', 'value']], how='left', on=['enc_id'])

In [ ]:
care_unit_df['tsp'] = pd.to_datetime(care_unit_df.tsp)
care_unit_df['end_time'] = pd.to_datetime(care_unit_df.end_time)
care_unit_df['admit_time'] = pd.to_datetime(care_unit_df.admit_time)

In [ ]:
care_unit_df = care_unit_df.rename(columns={'value':'care_unit'})

In [ ]:
care_unit_df = care_unit_df.merge(care_defs[['care_unit', 'level_of_care', 'pediatric', 'maternity']], how='left', on='care_unit')

In [ ]:
care_unit_df['level_of_care'] = care_unit_df.level_of_care.fillna('other')

In [ ]:
dep_df = care_unit_df.query('level_of_care != "icu" and tsp < end_time').sort_values(by=['enc_id', 'tsp']).groupby('enc_id', as_index=False).nth(-1)

In [ ]:
dep_df.query('level_of_care == "other" and enc_id in @ed_wrong_alert_ids').care_unit

In [ ]:
dep_df.query('enc_id in @ed_wrong_alert_ids and level_of_care == "surgery"').shape[0]/ed_wrong_alert_ids.shape[0]

In [ ]:
dep_df.query('enc_id in @ed_wrong_alert_ids')[['enc_id', 'hospital', 'level_of_care']].groupby(['hospital', 'level_of_care']).count()

In [ ]:
dep_df.query('enc_id in @ed_wrong_alert_ids and hospital != "JHH"')[['enc_id', 'hospital', 'level_of_care']].groupby(['hospital', 'level_of_care']).count().enc_id.plot(kind='barh', title='Last Department')

In [ ]:
dep_df[['enc_id', 'hospital', 'level_of_care']].groupby(['hospital', 'level_of_care']).count().enc_id.plot(kind='barh')

## First department

In [ ]:
first_df = care_unit_df.query('tsp >= admit_time').groupby('enc_id', as_index=False).nth(0)

In [ ]:
first_df

In [ ]:
first_df.query('enc_id in @ed_wrong_alert_ids and hospital != "JHH"')[['enc_id', 'hospital', 'level_of_care']].groupby(['hospital', 'level_of_care']).count().plot(kind='barh', title='First Department after ED')

In [ ]:
first_df.query('hospital != "JHH"')[['enc_id', 'hospital', 'level_of_care']].groupby(['hospital', 'level_of_care']).count().enc_id.plot(kind='barh', figsize=(6,6), title='First Department')

In [ ]:
event_department_info = first_df.rename(columns = {'care_unit':'first_unit', 'level_of_care':'first_level'})

In [ ]:
event_department_info = event_department_info[['enc_id', 'first_unit', 'first_level']]

In [ ]:
event_department_info = event_department_info.merge(dep_df[['enc_id', 'care_unit', 'level_of_care']], how='left', on='enc_id')

In [ ]:
event_department_info = event_department_info.rename(columns = {'care_unit':'last_unit', 'level_of_care':'last_level'})

In [ ]:
event_department_info.to_csv("/data/adarsh/fda_project_data/event_pt_care_unit_df.csv", index=False)

In [ ]:
surgery_df = cdm_t.query('fid == "surgery_proc"')

In [ ]:
json_df = pd.json_normalize(surgery_df.value.apply(json.loads))

In [ ]:
for c in json_df.columns:
    print(c)
    surgery_df[c] = json_df[c].values

In [ ]:
surgery_df

In [ ]:
pd.json_normalize(cdm_s.query('fid == "chief_complaint" and enc_id in @wrong_ed_alert_ids').value.apply(json.loads).apply(lambda x: x[0]))

In [ ]:
cdm_t.query('enc_id in @ed_wrong_alert_ids and fid == "surgery_proc"').value.apply(json.loads))